In [1]:
import nbimporter

# Import 1d well examples
from GPDM_kwells_simulate import *

plotly.offline.init_notebook_mode(connected=True)

from joblib import Parallel, delayed
import multiprocessing
import itertools

num_cores = multiprocessing.cpu_count()

# %run GPDM_direct_fixedpoints.ipynb
# %run GPDM_Examples.ipynb

Importing Jupyter notebook from GPDM_kwells_simulate.ipynb
Importing Jupyter notebook from GPDM_direct_fixedpoints.ipynb


/nfs/nhome/live/gbohner/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



# Experimental setup

1. We first generate training and test data with the same parameters.

2. We then fit GP models with a fixed number of parameters, but vary the number of fixed points versus basic inducing points

3. We compare the performance of the GP models against one-another as well as against linear autoregression.

## 1. Generate training and test data

In [2]:
kwells_params = OrderedDict()
kwells_params["rseed"] = 1234 # Happens to creates balanced training set in the first 8 trials in y
kwells_params["Ny"] = 500
kwells_params["T"] = 20
kwells_params["slope"] = -0.9
kwells_params["Sigma_eps"] = 5e-2
kwells_params["well_locs"] = np.array([1.0, -1.0])
kwells_params["well_width"] = 3
kwells_params["Sigma_nu"] = 1e-3
kwells_params["mu_0_0"] = 0.
kwells_params["Sigma_0_0"] = 1e-4


# Generate trials and collect them into [D x T x N] array
np.random.seed(kwells_params["rseed"])
all_trials_x = []
all_trials_y = []
for n in range(kwells_params["Ny"]):
    x,y = kwells_draw_trial(transform_func = (lambda x: x**3), **kwells_params)
    
    all_trials_x.append(x[:,:,None])
    all_trials_y.append(y[:,:,None])
    
x = np.concatenate(all_trials_x, axis=2)
y = np.concatenate(all_trials_y, axis=2)

# Swap around trials to ensure balanced training sets
cur_trial = 0
cur_trial_type = 1.
while cur_trial < 0.4*kwells_params["Ny"]:
    # If current trial is not correct type, switch it with one that is
    if not (np.mean(y[:,(kwells_params["T"]-4):,cur_trial]) * cur_trial_type > 0.2):
        switch_trial = cur_trial+1
        while (not (np.mean(y[:,(kwells_params["T"]-4):,switch_trial]) * cur_trial_type > 0.2)):
            switch_trial += 2
            
        tmp = np.copy(y[:,:,switch_trial])
        y[:,:,switch_trial] = np.copy(y[:,:,cur_trial])
        y[:,:,cur_trial] = tmp
        
        tmp = np.copy(x[:,:,switch_trial])
        x[:,:,switch_trial] = np.copy(x[:,:,cur_trial])
        x[:,:,cur_trial] = tmp
        
    #print np.mean(y[:,(kwells_params["T"]-4):,cur_trial]) * cur_trial_type
    cur_trial += 1
    cur_trial_type *= -1.

plots_by_run = []
for v in range(128):
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(kwells_params["T"])), 
                      y=np.squeeze(x[:,:,v]), 
                      mode='lines')
    )
    
    
print rmse(pred_lin_AR1(y[:,:,200:], y[:,:,:50], cutoff=None), y[:,:,200:])
    
    
xstar = np.atleast_2d(np.arange(-2.5,2.5,0.05))
true_tr_vals = kwells_true_tr_fnc(xstar, **kwells_params)

#set_trace()

plt([plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange')),
 plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals+np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange', size=2)),
 plt_type.Scatter(x=np.squeeze(xstar), 
                  y=np.squeeze(true_tr_vals-np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                  marker=dict(color='orange', size=2))
     ])

plt(plots_by_run)

0.365501842329


## 3. Load and compare fits

In [3]:
start_time = '20180129T144609'
nbatches = 1
ntrials_set = np.array([8, 16])
nfix_set = np.array([1,2,3,4,5])

# start_time = '20180119T185848'
# nbatches = 4 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180119T121802'
# nbatches = 2
# ntrials_set = np.array([8, 16])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180118T224743'
# nbatches = 2
# ntrials_set = np.array([8, 128])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180118T174510'
# nbatches = 2
# ntrials_set = np.array([2, 4, 8])
# nfix_set = np.array([0,2,4,5])


# start_time = '20180118T174510'
# nbatches = 2
# ntrials_set = np.array([2, 4, 8])
# nfix_set = np.array([0,2,4,5])

# start_time = '20180117T191020'
# nbatches = 10
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4,5])

# start_time = '20180117T032858'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16, 32])
# nfix_set = np.array([0,1,2,3,4,5])


# start_time = '20180105T145529'
# nbatches = 1
# ntrials_set = np.array([128])
# nfix_set = np.array([0,1,2,3,4])


# start_time = '20180105T145529'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4])

# start_time = '20180104T170845'
# nbatches = 10 
# ntrials_set = np.array([2, 4, 8, 16])
# nfix_set = np.array([0,1,2,3,4])

In [4]:
# Establish test dataset
y_test = y[:,:,350:]
x_test = x[:,:,350:]

In [5]:
# Load the saved files
def load_GP_fit(ntrials_ind, nfix_ind, batchnum_ind, ntrials, nfix, batchnum, start_time):
    load_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (nfix, ntrials, batchnum)
    load_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + load_fname_params + ".pkl"
    # load_fname_params = "_%2d_fix_%3d_trials" % (nfix, ntrials)
    # load_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + load_fname_params + ".pkl"

    with open(load_fname, 'r') as f:
        # Load and store results
        results_file = pickle.load(f)
    
    return results_file

GP_fit_saves = Parallel(n_jobs=num_cores)(
    delayed(load_GP_fit)(ntrials_ind, nfix_ind, batchnum_ind, ntrials, nfix, batchnum, start_time)
    for (ntrials_ind, nfix_ind, batchnum_ind), (ntrials, nfix, batchnum) in (
        itertools.izip(
            itertools.product(range(len(ntrials_set)), range(len(nfix_set)), range(nbatches)),
            itertools.product(ntrials_set, nfix_set, range(nbatches))
        )
    )
)

In [6]:
# Create predictions for each test set trial
def test_GP_pred(y_test, GPfit, cutoff=None):
    # Sort results into variables to work with
    [y_train, x_train, kwells_params,
                 all_results, 
                 init_paramvec, dict_ind, dict_shape, opt_params, 
                 bnds, transforms] = GPfit                                   
                                   
                                   
    # Get GP predictions on test data
    return pred_GP(y_test, 
                   replace_params(all_results[-1].x, opt_params, init_paramvec), 
                   transforms, dict_ind, dict_shape, cutoff = cutoff)


# Load or make and save predictions
cutoff = None
preds_fname = "Experiment_1d_wells_results/well_1d_k2_" + start_time + "_predictions.pkl"
try:
    [GP_predictions, ntrials_set, nfix_set, nbatches] = (
        pickle.load(open(preds_fname, 'rb')))
except (OSError, IOError) as e:
    GP_predictions = Parallel(n_jobs=num_cores)(
        delayed(test_GP_pred)(y_test, GPfit, cutoff=cutoff)
        for GPfit in GP_fit_saves
    )
    pickle.dump([GP_predictions, ntrials_set, nfix_set, nbatches], 
                open(preds_fname, 'wb'))


In [7]:
# Get errors based on predictions
axisError = (0,1,2)

AR_RMSE = []
for GPfit in GP_fit_saves:
    AR_RMSE.append(rmse(pred_lin_AR1(y_test, GPfit[0], cutoff=cutoff), y_test, axis=axisError))

GP_NLL = []
GP_RMSE = []
for GPpred in GP_predictions:
    GP_NLL.append(np.sum(GPpred[5], axis=tuple(np.array(axisError)[1:]-1)))
    GP_RMSE.append(rmse(GPpred[7], y_test, axis=axisError))
    

AR_RMSE = np.reshape(np.array(AR_RMSE), (len(ntrials_set), len(nfix_set), nbatches))
GP_RMSE = np.reshape(np.array(GP_RMSE), (len(ntrials_set), len(nfix_set), nbatches))
GP_NLL = np.reshape(np.array(GP_NLL), (len(ntrials_set), len(nfix_set), nbatches))


In [8]:
np.set_printoptions(precision=2)
print "AR_RMSE"
print np.array(np.median(AR_RMSE, axis=2))
print "-------------"
print "GP_RMSE"
print np.array(np.median(GP_RMSE, axis=2))
print "-------------"
print "GP_NLL"
print np.array(np.median(GP_NLL, axis=2))

AR_RMSE
[[ 0.37  0.37  0.37  0.37  0.37]
 [ 0.37  0.37  0.37  0.37  0.37]]
-------------
GP_RMSE
[[ 0.26  0.26  0.26  0.26  0.26]
 [ 0.25  0.25  0.25  0.25  0.27]]
-------------
GP_NLL
[[ 236.45  162.32  241.78  302.84  174.8 ]
 [  89.02  122.08  125.8    19.4   198.07]]


In [9]:
# Plot negative log likelihoods for each options
def plot_nll_fixpoints(cur_nll_ntrial, nfix_set, nbatches):
    cur_plts = []
    for (nfix_ind, nfix) in itertools.izip(range(len(nfix_set)), nfix_set):
        cur_plts.append(plt_type.Scatter(x=nfix*np.ones_like(np.squeeze(cur_nll_ntrial[nfix_ind,:])), 
                                         y=np.squeeze(cur_nll_ntrial[nfix_ind,:]), 
                                         mode='markers', 
                                         marker = dict(color=plotly.colors.DEFAULT_PLOTLY_COLORS[nfix_ind]),
                                         name = "nfix=%d" % nfix, 
                                         legendgroup="nfix=%d" % nfix))
    return cur_plts
    
    
from plotly import tools as plt_tools

def plot_nll(cur_nll, ntrials_set, nfix_set, nbatches):
    fig_subplts = plt_tools.make_subplots(rows=1, cols=len(ntrials_set), subplot_titles=ntrials_set, shared_yaxes=True)
    for (ntrials_ind, ntrials) in itertools.izip(range(len(ntrials_set)), ntrials_set):        
        tmp = plot_nll_fixpoints(cur_nll[ntrials_ind,:,:], nfix_set, nbatches)
        for trace in tmp:
            fig_subplts.append_trace(trace,1,ntrials_ind+1)
            if ntrials_ind>0:
                fig_subplts['data'][-1]['showlegend']=False
    plt(fig_subplts)
    return fig_subplts

a = plot_nll(GP_NLL, ntrials_set, nfix_set, nbatches)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [10]:
np.set_printoptions(precision=2)
print "-------------"
print "GP_NLL"
print np.array(np.min(GP_NLL, axis=2))
print "-------------"
print "GP_NLL"
print np.array(np.argmin(GP_NLL, axis=2))

-------------
GP_NLL
[[ 236.45  162.32  241.78  302.84  174.8 ]
 [  89.02  122.08  125.8    19.4   198.07]]
-------------
GP_NLL
[[0 0 0 0 0]
 [0 0 0 0 0]]


In [11]:
def nestedlist_to_array(nested_list, max_level = 0):
    cur_nested_list = nested_list
    level = 0
    array_dims = []
    array_dims_range = []
    while (level <= max_level) and (type(cur_nested_list)==list):
        array_dims.append(len(cur_nested_list))
        array_dims_range.append(range(len(cur_nested_list)))
        cur_nested_list = cur_nested_list[0]
        level += 1

    out = np.empty(tuple(array_dims), dtype=object)
    
    for inds in itertools.product(*array_dims_range):
        cur_nested_list = nested_list
        for i in inds:
            cur_nested_list = cur_nested_list[i]
        out[inds] = cur_nested_list
    
    return out

In [16]:
# Plot a single fit
GP_fit_saves_rs = np.reshape(nestedlist_to_array(GP_fit_saves, max_level=0), 
                             (len(ntrials_set), len(nfix_set), nbatches))

nfix_ind = 4
ntrials_ind = 1
batchnum = 0

print GP_RMSE[ntrials_ind, nfix_ind, batchnum]
print GP_NLL[ntrials_ind, nfix_ind, batchnum]

[y_train, x_train, kwells_params,
 all_results, 
 init_paramvec, dict_ind, dict_shape, opt_params, 
 bnds, transforms] = GP_fit_saves_rs[ntrials_ind, nfix_ind, batchnum]

print len(init_paramvec)

kwells_callback_plot_external(init_paramvec[opt_params], 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        kwells_params)

for res in all_results[:-1]:
    kwells_callback_plot_external(res.x, 
            opt_params, init_paramvec, transforms, dict_ind, dict_shape,
            kwells_params)
    
kwells_callback_plot_external(all_results[-1].x, 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        kwells_params)

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()
    
np.set_printoptions(precision=8)
print np.concatenate([s.T, Sigma_s], axis=1)

0.267547165203
198.069918218
60


[[-0.21789692  0.01645619]
 [-0.94159838  0.00525167]
 [ 0.74823915  0.01920249]
 [ 0.20483268  0.01402411]
 [ 1.08590231  0.00731822]]


In [ ]:
paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()

In [ ]:
len(all_results)

In [15]:
np.set_printoptions(precision=8)
paramdict

OrderedDict([('Sigma_eps', array([[ 0.04041312]])),
             ('mu_0_0', array([[ 0.01510288]])),
             ('Sigma_0_0', array([[ 0.01901289]])),
             ('C', array([[ 1.]])),
             ('Sigma_nu', array([[ 0.001]])),
             ('z',
              array([[-1.42607721, -1.19586547, -0.87915898, -0.57515666, -0.35406786,
                      -0.16249179,  0.17734388,  0.4154941 ,  0.70181852,  0.98172765,
                       1.24198829,  1.51956617]])),
             ('u',
              array([[-1.0996458 , -1.02929873, -0.98891408, -0.97689754, -0.10508028,
                       0.22042376, -0.21731689,  0.32505404,  1.01935796,  1.04831552,
                       0.9640638 ,  0.73011606]])),
             ('Sigma_u', array([[  2.09802076e-02],
                     [  9.10454831e-04],
                     [  1.13979775e-03],
                     [  1.00915159e-03],
                     [  1.50976416e-03],
                     [  7.85210206e-06],
                  

In [19]:
GP_predictions[1][6]

0

In [ ]:
vec_to_params(init_paramvec, dict_ind, dict_shape, transforms)

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u
print s
print Sigma_s
print Sigma_J

In [ ]:
print lengthscales

In [ ]:
# Add priors (to span at least the bounds)
priors = []

# Add prior to ensure inducing points are smooth
cur_prior = {}
cur_prior['func'] = create_prior(prior_distribution="InducingSmooth")
cur_prior['inds'] = range(len(init_paramvec))
cur_prior['metadata'] = {}
def unpack_x_tmp(x):
    # Unpack all the usual parameters
    param_tuple = vec_to_params(x, dict_ind, dict_shape)
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, 
     z, u, Sigma_u, 
     lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
        param_tuple

    kernelparams = {'lengthscales': lengthscales, 'kernel_variance': kernel_variance}
    # Return the ones we want in the required format
    return (np.concatenate([u, s],axis=1), 
            np.concatenate([z, s],axis=1),
            np.concatenate([Sigma_u, Sigma_s]),
            kernelparams)
cur_prior['metadata']['unpack_x'] = unpack_x_tmp
cur_prior['metadata']['kernel_func'] = RBF
priors.append(cur_prior)

tmp_func = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                                y_train, dict_ind, dict_shape, 
                                                log_transformed=log_transformed,
                                                priors=priors)[0])


In [ ]:
paramvec = replace_params(all_results[1].x, opt_params, init_paramvec)
#paramvec = log_transform_inv(paramvec, log_transformed)
paramvec = paramvec[opt_params]
a = grad(tmp_func)(paramvec)

In [ ]:
paramvec = replace_params(a, opt_params, init_paramvec)
#paramvec = log_transform_inv(paramvec, log_transformed)

# Unpack the usual parameters
param_tuple = vec_to_params(paramvec, dict_ind, dict_shape)
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    param_tuple

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u
print s
print Sigma_s

In [ ]:
np.set_printoptions(precision=8)
print z
print u
print Sigma_u

In [ ]:
print s
print Sigma_s

In [ ]:
vec_to_params(init_paramvec, dict_ind, dict_shape, transforms=transforms)